In [1]:
from renjuRL_DK.renju_game.brains.BoardManager import B_SHAPE, PLAYERS

In [22]:
label2pl = {
    'black': PLAYERS[0],
    'white': PLAYERS[1],
    'draw': 0,
    'unknown': None
}

In [20]:
def refresh_board():
    board = np.dstack((
            np.zeros((15, 15, 2)),
            np.ones((15, 15, 1))
        ))
        
    # board[x, y, 2] <- 0 if we made step (x, y)
    # --//--         <- 1/-1 if it is allowed to move
    # Value means which player are going to move
    step_mask = np.full((15, 15, 1), True)
    next_player = PLAYERS[0]
    
    return board, step_mask, next_player

def data_generator(parties, labels, batch_size, data_mask):
    assert batch_size > 0, "Negative batch size was passed"
    
    p_num = len(data_mask)
    
    s_idx = p_idx = 0
    
    result = label2pl[labels[data_mask[p_idx]]]
    while result is None:
        p_idx += 1
        if p_idx >= p_num:
            p_idx = 0
            
        result = label2pl[labels[data_mask[p_idx]]]
        board, step_mask, next_player = refresh_board()       
    while True:
        batch = []
        moves = []
        
        bch_iter = 0
        while bch_iter < batch_size:
            if s_idx >= len(parties[data_mask[p_idx]]):
                board, step_mask, next_player = refresh_board()
                
                p_idx += 1
                s_idx = 0
                if p_idx >= p_num:
                    p_idx = 0
                    
                result = label2pl[labels[data_mask[p_idx]]]
                while result is None:
                    p_idx += 1
                    if p_idx >= p_num:
                        p_idx = 0
            
                    result = label2pl[labels[data_mask[p_idx]]]   
            # -----------------------------------------
            
            move = parties[data_mask[p_idx]][s_idx]
            x, y = move2cord(move)

            if next_player != -result:
                batch.append(board)
                moves.append(x * 15 + y)
                bch_iter += 1
                
            if x != 15:
                # ((3 - player) mod 3) mod 2 gives correct board channel
                board[x, y, ((3-next_player) % 3) % 2] = next_player
                board[x, y, 2] = 0
            
            next_player = -next_player
            
        yield np.array(batch), to_categorical(moves, 226)

In [21]:
np.random.choice(np.arange(10000), 100)

array([1083, 9365, 3067, 6800, 3518, 9553, 8608, 1616, 5851, 4537, 5717,
       8295, 8693, 8150, 1759, 7894, 3938, 2897, 8497, 2359, 2981, 3588,
        568, 1952, 6884, 4748, 5316, 5002, 6727, 8202, 9762, 4136, 3001,
       5868, 2240, 1384, 8953, 6725, 4508, 2176, 6492, 9490, 4789,  336,
       1319, 6951,  927, 9298, 2174, 5176, 5370, 5184, 1486, 3746, 8991,
        184, 1035, 8135,  289, 7378, 7016,  568, 4696, 2464, 8446, 8860,
       9281, 2154, 9944, 7768,   76, 7916,    2, 9212, 1609, 7589, 6716,
       1145, 5311, 9981,  848, 2091, 3135, 4789, 6502,  276, 3197, 5158,
       5129, 1033, 6997, 3850, 2485, 1020, 5882, 4943, 8925, 4061, 8450,
       4939])

In [22]:
label2pl = {
    'black': PLAYERS[0],
    'white': PLAYERS[1],
    'draw': 0,
    'unknown': None
}

def move2cord(move):
    if move == '--':
        return 15, 0
    
    move = (move[0], move[1:])
    x = X_MAPPING[move[1]]
    y = Y_MAPPING[move[0]]
    
    return x, y

def refresh_board():
    board = np.dstack((
            np.zeros((15, 15, 2), dtype=np.int8),
            np.ones((15, 15, 1), dtype=np.int8)
        ))
        
    # board[x, y, 2] <- 0 if we made step (x, y)
    # --//--         <- 1/-1 if it is allowed to move
    # Value means which player are going to move
    step_mask = np.dstack((
        np.full((15, 15, 2), False),
        np.full((15, 15, 1), True)
    ))
    next_player = PLAYERS[0]
    
    return board, step_mask, next_player

def data_generator(parties, labels, batch_size, data_mask):
    assert batch_size > 0, "Negative batch size was passed"
    
    p_num = len(data_mask)
    
    s_idx = p_idx = 0
    
    result = label2pl[labels[data_mask[p_idx]]]
    while result is None:
        p_idx += 1
        if p_idx >= p_num:
            p_idx = 0
            
    result = label2pl[labels[data_mask[p_idx]]]
    board, step_mask, next_player = refresh_board()       
    while True:
        batch = []
        moves = []
        
        bch_iter = 0
        while bch_iter < batch_size:
            if s_idx >= len(parties[data_mask[p_idx]]):
                board, step_mask, next_player = refresh_board()
                
                p_idx += 1
                s_idx = 0
                if p_idx >= p_num:
                    p_idx = 0
                    
                result = label2pl[labels[data_mask[p_idx]]]
                while result is None:
                    p_idx += 1
                    if p_idx >= p_num:
                        p_idx = 0
            
                    result = label2pl[labels[data_mask[p_idx]]]   
            # -----------------------------------------
            
            move = parties[data_mask[p_idx]][s_idx]
            s_idx += 1
            x, y = move2cord(move)

            if next_player != -result:
                if x != 15:
                    board[step_mask] = next_player
                batch.append(board.copy())
                moves.append(x * 15 + y)
                bch_iter += 1
                
            if x != 15:
                # ((3 - player) mod 3) mod 2 gives correct board channel
                board[x, y, ((3-next_player) % 3) % 2] = next_player
                board[x, y, 2] = 0
                step_mask[x, y, 2] = False
            
            next_player = -next_player
            
        yield np.array(batch), to_categorical(moves, 226)

NameError: name 'PLAYERS' is not defined